# Construyendo Agentes de IA con Memoria Persistente usando Cognee

Este cuaderno demuestra cómo construir agentes de IA inteligentes con capacidades avanzadas de memoria utilizando [**cognee**](https://www.cognee.ai/) - una memoria de IA de código abierto que combina gráficos de conocimiento, búsqueda semántica y gestión de sesiones para crear sistemas de IA conscientes del contexto.

## 🎯 Objetivos de Aprendizaje

Al final de este tutorial, comprenderás cómo:
- **Construir Gráficos de Conocimiento Respaldados por Embeddings**: Transformar texto no estructurado en conocimiento estructurado y consultable
- **Implementar Memoria de Sesión**: Crear conversaciones de múltiples turnos con retención automática de contexto
- **Persistir Conversaciones**: Almacenar opcionalmente interacciones importantes en memoria a largo plazo para referencia futura
- **Consultar Usando Lenguaje Natural**: Acceder y aprovechar el contexto histórico en nuevas conversaciones
- **Visualizar Memoria**: Explorar las relaciones en el gráfico de conocimiento de tu agente


## 🏗️ Lo que Construirás

En este tutorial, crearemos un **Asistente de Programación** con memoria persistente que:

### 1. **Construcción de una Base de Conocimientos**
   - Ingiera información del perfil y experiencia del desarrollador
   - Procese principios y mejores prácticas de programación en Python
   - Almacene conversaciones históricas entre desarrolladores y asistentes de IA

### 2. **Conversaciones Conscientes de la Sesión**
   - Mantenga el contexto a lo largo de múltiples preguntas en la misma sesión
   - Almacene automáticamente cada par de pregunta/respuesta para una recuperación eficiente
   - Proporcione respuestas coherentes y contextuales basadas en el historial de la conversación

### 3. **Memoria a Largo Plazo**
   - Persista conversaciones importantes en una memoria a largo plazo
   - Recupere recuerdos relevantes de la base de conocimientos y sesiones pasadas para informar nuevas interacciones
   - Construya una base de conocimientos en crecimiento que mejore con el tiempo

### 4. **Recuperación Inteligente de Memoria**
   - Utilice búsqueda semántica basada en grafos para encontrar información relevante en todo el conocimiento almacenado
   - Filtre búsquedas por subgrupos de datos (información del desarrollador vs. principios)
   - Combine múltiples fuentes de datos para proporcionar respuestas completas


## 📋 Requisitos previos y configuración

### Requisitos del sistema

Antes de comenzar, asegúrate de tener:

1. **Entorno de Python**
   - Python 3.9 o superior
   - Entorno virtual (recomendado)
   
2. **Cache de Redis** (Requerido para la gestión de sesiones)
   - Redis local: `docker run -d -p 6379:6379 redis`
   - O utiliza un servicio gestionado de Redis
   
3. **Acceso a la API de LLM**
   - Clave de API de OpenAI u otros proveedores (consulta la [documentación](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Configuración de la base de datos**
   - No se requiere configuración por defecto. Cognee utiliza bases de datos basadas en archivos (LanceDB y Kuzu)
   - Opcionalmente, puedes configurar Azure AI Search como un almacén vectorial (consulta la [documentación](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Configuración del entorno

Crea un archivo `.env` en el directorio de tu proyecto con las siguientes variables:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Comprendiendo la Arquitectura de Memoria de Cognee

### Cómo Funciona Cognee

Cognee ofrece un sistema de memoria sofisticado que va más allá del simple almacenamiento de clave-valor:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Componentes Clave:

1. **Grafo de Conocimiento**: Almacena entidades, relaciones y conexiones semánticas
2. **Embeddings Vectoriales**: Permite la búsqueda semántica en toda la información almacenada
3. **Caché de Sesión**: Mantiene el contexto de la conversación dentro y entre sesiones
4. **NodeSets**: Organiza los datos en categorías lógicas para una recuperación específica

### Tipos de Memoria en Este Tutorial:

- **Memoria Persistente**: Almacenamiento a largo plazo en el grafo de conocimiento
- **Memoria de Sesión**: Contexto temporal de la conversación en la caché de Redis
- **Memoria Semántica**: Búsqueda basada en similitud vectorial en todos los datos


## 📦 Instalar paquetes requeridos

Instala Cognee con soporte de Redis para la gestión de sesiones:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Inicializar el Entorno y Cargar Bibliotecas

Asegúrate de que:
1. Redis esté en funcionamiento (por ejemplo, usando Docker: `docker run -d -p 6379:6379 redis`)
2. Las variables de entorno estén configuradas antes de importar los módulos de caché
3. Si es necesario, reinicia el kernel y ejecuta las celdas en orden

La siguiente celda hará lo siguiente:
1. Cargar las variables de entorno desde `.env`
2. Configurar Cognee con tus ajustes de LLM
3. Habilitar la caché para la gestión de sesiones
4. Validar que todos los componentes estén correctamente conectados


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Configurar Directorios de Almacenamiento

Cognee utiliza dos directorios separados para sus operaciones:
- **Raíz de Datos**: Almacena documentos ingeridos y datos procesados
- **Raíz del Sistema**: Contiene la base de datos del grafo de conocimiento y los metadatos del sistema

Crearemos directorios aislados para este tutorial de la siguiente manera:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Restablecer el Estado de la Memoria

Antes de comenzar a construir nuestro sistema de memoria, asegurémonos de empezar desde cero.

> 💡 **Consejo**: Puedes omitir este paso si deseas conservar los recuerdos existentes de ejecuciones anteriores cuando uses este cuaderno más adelante.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Parte 1: Construyendo la Base de Conocimiento

### Fuentes de Datos para Nuestro Asistente de Desarrolladores

Incorporaremos tres tipos de datos para crear una base de conocimiento integral:

1. **Perfil del Desarrollador**: Experiencia personal y antecedentes técnicos
2. **Mejores Prácticas de Python**: El Zen de Python con pautas prácticas
3. **Conversaciones Históricas**: Sesiones de preguntas y respuestas pasadas entre desarrolladores y asistentes de IA

Estos datos diversos permiten que nuestro agente:
- Comprenda el contexto técnico del usuario
- Aplique mejores prácticas en sus recomendaciones
- Aprenda de interacciones exitosas anteriores


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Procesar Datos en un Grafo de Conocimiento

Ahora transformaremos nuestro texto sin procesar en una memoria estructurada. Este proceso:

1. **Agrega datos a NodeSets**: Organiza la información en categorías lógicas
   - `developer_data`: Perfil del desarrollador y conversaciones
   - `principles_data`: Mejores prácticas y directrices de Python

2. **Ejecuta el Pipeline Cognify**: Extrae entidades, relaciones y crea embeddings
   - Identifica conceptos clave
   - Crea conexiones semánticas entre información relacionada
   - Genera embeddings vectoriales

Esto puede tomar unos momentos mientras el LLM procesa el texto y construye la estructura del grafo:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Visualizar el Grafo de Conocimiento

Vamos a explorar la estructura de nuestro grafo de conocimiento. La visualización muestra:
- **Nodos**: Entidades extraídas del texto (conceptos, tecnologías, personas)
- **Aristas**: Relaciones y conexiones entre las entidades
- **Clústeres**: Conceptos relacionados agrupados por similitud semántica

Abre el archivo HTML generado en tu navegador para explorar el grafo de manera interactiva:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Enriquecer la Memoria con Memify

La función `memify()` analiza el grafo de conocimiento y genera reglas inteligentes sobre los datos. Este proceso:
- Identifica patrones y mejores prácticas
- Crea pautas accionables basadas en el contenido
- Establece relaciones entre diferentes áreas de conocimiento

Estas reglas ayudan al agente a tomar decisiones más informadas al responder preguntas. Capturar una segunda visualización te permite comparar cómo se densifica el grafo una vez enriquecido.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Parte 2: Recuperación Inteligente de Memoria

### Demostración 1: Integración de Conocimientos entre Documentos

Ahora que nuestro grafo de conocimiento está construido, probemos cómo Cognee combina información de múltiples fuentes para responder preguntas complejas.

La primera consulta demuestra:
- **Comprensión semántica**: Encontrar conceptos relevantes incluso cuando no se mencionan explícitamente
- **Referencia cruzada**: Combinar el perfil del desarrollador con principios de Python
- **Razonamiento contextual**: Aplicar mejores prácticas a implementaciones específicas

### Demostración 2: Búsqueda Filtrada con NodeSets

La segunda consulta muestra cómo dirigirnos a subconjuntos específicos del grafo de conocimiento:
- Utiliza el parámetro `node_name` para buscar solo dentro de `principles_data`
- Proporciona respuestas enfocadas de un dominio de conocimiento específico
- Útil cuando necesitas información específica de un dominio


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Parte 3: Configuración de Gestión de Sesiones

### Habilitar Memoria de Conversación

La gestión de sesiones es crucial para mantener el contexto a lo largo de múltiples interacciones. Aquí haremos lo siguiente:

1. **Inicializar el Contexto del Usuario**: Crear o recuperar un perfil de usuario para el seguimiento de la sesión.
2. **Configurar el Motor de Caché**: Conectar a Redis para almacenar el historial de conversaciones.
3. **Habilitar Variables de Sesión**: Configurar variables de contexto que persistan entre consultas.

> ⚠️ **Importante**: Esto requiere que Redis esté en funcionamiento y que `CACHING=true` esté configurado en tu entorno.


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Función de Ayuda: Ver Historial de Sesión

Esta función de utilidad nos permite inspeccionar el historial de conversaciones almacenado en Redis. Es útil para:
- Depurar la gestión de sesiones
- Verificar que las conversaciones se están almacenando en caché
- Comprender qué contexto está disponible para el agente


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Sesión 1: Laboratorio de soporte asíncrono — Primera pregunta

Inicia la sesión `async-support-lab` preguntando por patrones de asyncio compatibles con telemetría para un scraper web masivo. El gráfico ya tiene conocimiento sobre asyncio, aiohttp y prácticas de monitoreo, por lo que la respuesta debería reflejar conversaciones previas mientras adapta la respuesta a la nueva consulta.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Inspeccionar la Memoria de la Sesión 1 Después del Primer Intercambio

Ejecutar `show_history(session_1)` inmediatamente después de la pregunta inicial confirma que Cognee escribió tanto el mensaje inicial como la respuesta en Redis. Deberías ver una entrada con la guía de concurrencia.


In [ ]:
await show_history(session_1)

## Sesión 1: Seguimiento de Modelos de Datos

A continuación, preguntamos: "¿Cuándo debería elegir dataclasses en lugar de Pydantic?" utilizando el mismo id de sesión. Cognee debería combinar los principios de Python junto con conversaciones previas sobre FastAPI para ofrecer un consejo matizado, demostrando que el contexto se mantiene dentro de una sesión nombrada.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Confirmar que el Historial de la Sesión 1 Contiene Ambos Turnos

Otra llamada a `show_history(session_1)` debería mostrar dos entradas de preguntas y respuestas. Esto coincide con el paso de "reproducción de memoria" del laboratorio Mem0 y demuestra que los turnos adicionales amplían la misma transcripción.


In [ ]:
await show_history(session_1)

## Sesión 2: Hilo de Revisión de Diseño — Sesión Nueva

Para mostrar aislamiento entre hilos, iniciamos `design-review-session` y pedimos orientación sobre el registro para revisiones de incidentes. Aunque la base de conocimiento subyacente es la misma, el nuevo id de sesión mantiene las transcripciones separadas.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Revisión de la Sesión 2 Historia

`show_history(session_2)` debería listar únicamente el par de solicitud/respuesta de revisión de diseño. Compáralo con la Sesión 1 para destacar cómo Cognee mantiene transcripciones independientes mientras reutiliza el gráfico de conocimiento compartido.


In [ ]:
await show_history(session_2)

## Resumen

¡Felicidades! Acabas de darle a tu asistente de codificación una capa de memoria a largo plazo impulsada por Cognee.

En este tutorial, tomaste contenido de desarrollador en bruto (código, documentos, chats) y lo transformaste en un grafo + memoria vectorial que tu agente puede buscar, razonar y mejorar continuamente.

Lo que has aprendido

1. **De texto en bruto a memoria de IA**: Cómo Cognee ingiere datos no estructurados y los convierte en una memoria inteligente y buscable utilizando una arquitectura combinada de vector + grafo de conocimiento.

2. **Enriquecimiento del grafo con memify**: Cómo ir más allá de la creación básica de grafos y usar memify para agregar hechos derivados y relaciones más ricas sobre tu grafo existente.

3. **Múltiples estrategias de búsqueda**: Cómo consultar la memoria con diferentes tipos de búsqueda (preguntas y respuestas conscientes del grafo, completado estilo RAG, insights, fragmentos en bruto, búsqueda de código, etc.) dependiendo de lo que necesite tu agente.

4. **Exploración visual**: Cómo inspeccionar y depurar lo que Cognee construyó utilizando visualizaciones de grafos y la interfaz de usuario de Cognee, para que realmente puedas ver cómo está estructurado el conocimiento.

5. **Memoria consciente de la sesión**: Cómo combinar el contexto por sesión con la memoria semántica persistente para que los agentes puedan recordar entre ejecuciones sin filtrar información entre usuarios.


## Puntos Clave
1. Memoria como un Grafo de Conocimiento respaldado por Embeddings

    - **Comprensión estructurada**: Cognee combina un almacén vectorial y un almacén de grafos para que tus datos sean tanto buscables por significado como conectados por relaciones. Cognee utiliza bases de datos basadas en archivos por defecto (LanceDB para vectores, Kuzu para bases de datos de grafos).

    - **Recuperación consciente de relaciones**: Las respuestas pueden estar fundamentadas no solo en "texto similar", sino también en cómo se relacionan las entidades.

    - **Memoria viva**: La capa de memoria evoluciona, crece y permanece consultable como un grafo conectado.

2. Modos de Búsqueda y Razonamiento
    - **Recuperación híbrida**: La búsqueda combina similitud vectorial, estructura de grafos y razonamiento de LLM, desde la búsqueda de fragmentos hasta respuestas a preguntas conscientes del grafo.

    - **Ajusta el modo al trabajo**: Usa modos de estilo de completado cuando necesites respuestas en lenguaje natural, y modos de fragmento/resumen/grafo cuando tu agente necesite contexto bruto o impulsar su propio razonamiento.

3. Agentes Personalizados y Conscientes de la Sesión
    - **Contexto de sesión + memoria a largo plazo**: Cognee mantiene el contexto de "hilo" a corto plazo separado de la memoria duradera a nivel de usuario u organización.

## Aplicaciones en el Mundo Real

1. **Agentes Verticales de IA**

    Usa el patrón de este cuaderno para potenciar copilotos inteligentes en dominios que se basen en Cognee como su núcleo de recuperación y razonamiento:

- **Copilotos para desarrolladores**: Asistentes de revisión de código, análisis de incidentes y arquitectura que recorren código, APIs, documentos de diseño y tickets como un único grafo de memoria.

- **Copilotos orientados al cliente**: Agentes de soporte o éxito que extraen información de documentos de productos, preguntas frecuentes, notas de CRM y tickets anteriores con recuperación consciente del grafo y respuestas citadas.

- **Copilotos expertos internos**: Asistentes de políticas, legales o de seguridad que razonan sobre reglas interconectadas, guías y decisiones históricas en lugar de PDFs aislados.

    Cognee está explícitamente posicionado como memoria persistente y precisa para agentes de IA, proporcionando un grafo de conocimiento vivo que se integra detrás de tu agente y reemplaza combinaciones ad-hoc de almacenes vectoriales y código de grafos personalizado.

2. **Unificar Silos de Datos en Una Memoria**

    El mismo enfoque también te ayuda a construir una capa de memoria unificada a partir de fuentes dispersas:

- **De silos a un grafo único**: Ingresa datos estructurados (por ejemplo, bases de datos) y no estructurados (por ejemplo, documentos, chats) en un único grafo respaldado por embeddings, en lugar de índices separados para cada sistema.

- **Razonamiento entre fuentes con citas**: Realiza razonamientos de múltiples pasos sobre todo—"une" registros, métricas y documentos a través del grafo—y aún devuelve respuestas fundamentadas con procedencia.

- **Centros de conocimiento**: Para dominios como banca o educación, Cognee ya se utiliza para unificar PDFs, sistemas internos y datos de aplicaciones en un grafo de conocimiento con vectores para que los agentes puedan responder preguntas con contexto preciso y citado.

## Próximos Pasos

Has implementado el bucle de memoria principal. Aquí hay extensiones naturales que puedes probar por tu cuenta (consulta la [documentación de Cognee](https://docs.cognee.ai/) para más detalles):

1. **Experimenta con conciencia temporal**: Activa la cognificación temporal para extraer eventos y marcas de tiempo del texto.

2. **Introduce razonamiento basado en ontologías**: Define una ontología OWL para tu dominio. Usa el soporte de ontologías de Cognee para que las entidades y relaciones extraídas estén fundamentadas en ese esquema, mejorando la calidad del grafo y las respuestas específicas del dominio.

3. **Añade un bucle de retroalimentación**: Permite que Cognee ajuste los pesos de los bordes del grafo a partir de la retroalimentación real de los usuarios, para que la recuperación mejore con el tiempo en lugar de permanecer estática.

4. **Optimiza para personalización y comportamiento de sesión**: Usa IDs de usuario, inquilinos y conjuntos de datos para dar a cada persona o equipo su propia vista sobre el motor de memoria compartido.

5. **Escala hacia agentes más complejos**: Conecta Cognee a marcos de agentes para construir sistemas multi-agente que compartan la misma capa de memoria. *El plugin Microsoft Agent Framework x Cognee estará disponible pronto.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por lograr precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que surjan del uso de esta traducción.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
